In [1]:
%pip install google-cloud-bigquery
%pip install "google-cloud-bigquery[pandas]"
%pip install db-dtypes


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/tanya/Desktop/Data Science Journey/SQL/spheric-passkey-475414-m3-613464412c9f.json"

Let's work with the hacker_news dataset

In [3]:
#we need to create a client object to retrieve information from BigQuery datasets
client = bigquery.Client(project="spheric-passkey-475414-m3")  # ← your project ID

In [4]:
# datasets exist in projects in bigquery. we would like to work with hacker_news dataset contained in bigquery-public-data project
# here we connect to the dataset
hacker_ref = client.dataset("hacker_news", project="bigquery-public-data")


# now let's fetch the dataset
hacker_data = client.get_dataset(hacker_ref)

returning the tables

In [5]:
# this dataset is like a set of tables. so now let's list those tables 
hacker_tables = list(client.list_tables(hacker_data))

# now we want the names of those tables. each of them has an id and this code shows the names of the tables
for table in hacker_tables:
    print(table.table_id)

# there is one table!

full


In [6]:
# Let's open a table now by calling it
# first we construct a reference to the table
hacker_full_table_ref = hacker_ref.table("full")

# then through that reference we fetch the table
hacker_full_table = client.get_table(hacker_full_table_ref)


In [7]:
# let's investigate the table. 
# print information on all the columns

hacker_full_table.schema

#each schemafield holds into about a specific column. 
# first is column name
# second is data type 
# third os mode of the column (nullable means allows null values)
# fourth is description of the data of the column 

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', N

In [8]:
# let's have a look at the first five rows of the full table

client.list_rows(hacker_full_table, max_results = 5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,True,Adoum_Tech,2,1713995025,2024-04-24 21:43:45+00:00,story,40150086,<NA>,<NA>,<NA>,<NA>
1,None,None,None,True,belter,2,1713995286,2024-04-24 21:48:06+00:00,story,40150135,<NA>,<NA>,<NA>,<NA>
2,None,None,None,True,Rinzler89,1,1713995678,2024-04-24 21:54:38+00:00,story,40150207,<NA>,<NA>,<NA>,<NA>
3,None,None,None,True,stockstobuynow,1,1713995704,2024-04-24 21:55:04+00:00,story,40150212,<NA>,<NA>,<NA>,<NA>
4,None,None,None,True,FLMAN407,1,1713995772,2024-04-24 21:56:12+00:00,story,40150229,<NA>,<NA>,<NA>,<NA>


In [9]:
# the method above also lets us to have information of a specific column
client.list_rows(hacker_full_table, selected_fields = hacker_full_table.schema[:1], max_results = 5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


Group by, Count

Which Hacker News comments generated the most discussion?

In [10]:
#parent is the column where there are comments that have been replied to

query_popular = """
            select parent, count(id)
            from `bigquery-public-data.hacker_news.full`
            group by parent 
            having count(id) > 10
            """

In [11]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query_popular, job_config=safe_config)

popular_comments = query_job.to_dataframe()

popular_comments.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,parent,f0_
0,34324567,299
1,34412610,97
2,34415704,48
3,34488938,65
4,34497898,42


Aliasing

In [12]:
# count(1) just counts rows in each group. it does not focus on any particular column so it is computationally more efficient

query_popular_improved = """
                        select parent, count(1) as NumPosts
                        from `bigquery-public-data.hacker_news.full`
                        group by parent
                        having count(1) > 10
                        """

In [13]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query_popular_improved, job_config=safe_config)

popular_comments_improved = query_job.to_dataframe()

popular_comments_improved.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,parent,NumPosts
0,9044030,47
1,9044719,44
2,9080665,41
3,9092584,57
4,9138728,77


Let's work with OpenAQ dataset 

In [14]:
dataset_ref_aq = client.dataset("openaq", project = "bigquery-public-data") #build a reference to the dataset

dataset_aq = client.get_dataset(dataset_ref_aq)  #fetch the dataset

tables_aq = list(client.list_tables(dataset_aq))  #list all the tables in openaq dataset

tabs = []
for table in tables_aq: #print the names of the tables
    tabs.append(table.table_id)

print(f"There are {len(tabs)} tables, and their names are: {tabs}")
    

There are 1 tables, and their names are: ['global_air_quality']


Now let's return the table columns and rows 

In [15]:
# first let's create a table reference
table_ref_aq = dataset_ref_aq.table("global_air_quality")

# fetch the table through reference of the table
table_aq_gaq = client.get_table(table_ref_aq)

#now let's list the rows
client.list_rows(table_aq_gaq, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


let's fetch the cities in the united states (where)

In [16]:
#let's write our query
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

# let's add more columns 
query2 = """
        select city, country
        from `bigquery-public-data.openaq.global_air_quality`
        where country = 'US'
        """

In [17]:
# We need to use the query to return what we want 
query_job = client.query(query2)

# query_job is the subset of the data we filtered and now we turn it into a pandas dataframe called us_cities
us_cities = query_job.to_dataframe()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [18]:
# let's get the value counts of the dataframe
us_cities.city.value_counts().head()

city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [19]:
us_cities

,city,country
0,HOWARD,US
1,HOWARD,US
2,HOWARD,US
3,HOWARD,US
4,HOWARD,US
...,...,...
1421346,Washington-Arlington-Alexandria,US
1421347,Washington-Arlington-Alexandria,US
1421348,Washington-Arlington-Alexandria,US
1421349,Washington-Arlington-Alexandria,US


Date format: YYYY-MM-DD <Br>
Datetime format: YYYY-MM-DD HH:MI:SS

EXTRACT

In [20]:
import pandas as pd

df_list = [[1, "Dr.Harris Bonkers", "Rabbit", "2019-04-18", 2], [4, "Tom", "Cat", "2019-05-16", 5], 
           [2, "Moon", "Dog", "2019-01-07", 7], [3, "Ripley", "Cat", "2019-02-23", 4]]

df = pd.DataFrame(df_list, columns=['ID', "Name", "Animal", "Date", "Years_old"])
df["Date"] = pd.to_datetime(df["Date"])
df.style.hide()      #hides the automatic indexing 

ID,Name,Animal,Date,Years_old
1,Dr.Harris Bonkers,Rabbit,2019-04-18 00:00:00,2
4,Tom,Cat,2019-05-16 00:00:00,5
2,Moon,Dog,2019-01-07 00:00:00,7
3,Ripley,Cat,2019-02-23 00:00:00,4


In [21]:
%pip install duckdb


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import duckdb

# Extracts day of the date and creates a column for those values
anm_query = """
            select Name, Extract(day from Date) as Day
            from df
            """

result = duckdb.query(anm_query).to_df()
print(result)

                Name  Day
0  Dr.Harris Bonkers   18
1                Tom   16
2               Moon    7
3             Ripley   23


In [23]:
# Extracts week of that date (which is not part of the cell) so it can find which week of the year that is 
anm_query2 = """
            select Name, Extract(week from Date) as Week
            from df
            """

result2 = duckdb.query(anm_query2).to_df()
print(result2)

                Name  Week
0  Dr.Harris Bonkers    16
1                Tom    20
2               Moon     2
3             Ripley     8


Join: creating a new table combining information from the pets and owners tables

In [24]:
import pandas as pd

# owners table
df_owners = pd.DataFrame({
    "ID": [1, 2, 3, 4],
    "Name": ["Aubrey Little", "Chett Crawfish", "Jules Spinner", "Magnus Burnside"],
    "Pet_ID": [1, 3, 4, 2]
})

# pets table
df_pets = pd.DataFrame({
    "ID": [1, 2, 3, 4],
    "Name": ["Dr. Harris Bonkers", "Moon", "Ripley", "Tom"],
    "Animal": ["Rabbit", "Dog", "Cat", "Cat"]
})

print(df_owners)
print(df_pets)

   ID             Name  Pet_ID
0   1    Aubrey Little       1
1   2   Chett Crawfish       3
2   3    Jules Spinner       4
3   4  Magnus Burnside       2
   ID                Name  Animal
0   1  Dr. Harris Bonkers  Rabbit
1   2                Moon     Dog
2   3              Ripley     Cat
3   4                 Tom     Cat


In [25]:
query_join = """
SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
FROM df_pets AS p
INNER JOIN df_owners AS o
    ON p.ID = o.Pet_ID
"""

result3 = duckdb.query(query_join).to_df()
print(result3)

             Pet_Name       Owner_Name
0  Dr. Harris Bonkers    Aubrey Little
1                Moon  Magnus Burnside
2              Ripley   Chett Crawfish
3                 Tom    Jules Spinner


INNER JOIN: a row will only be put in the final output table if the value in the columns you're using to combine them shows up in both the tables you're joining.

CTE (Common Table Expression)

In [26]:
query_cte = """
            WITH Seniors AS
            (
                SELECT ID, Name
                FROM df
                WHERE Years_old > 5
            )
            SELECT *
            FROM Seniors
            """

In [27]:
result3 = duckdb.query(query_cte).to_df()
print(result3)

   ID  Name
0   2  Moon


#### LEFT JOIN, RIGHT JOIN, FULL JOIN, and UNIONs <br>
LEFT JOIN returns all rows where the two tables have matching entries, along with all of the rows in the left table (whether there is a match or not). <br> <br>
In a RIGHT JOIN, we get the matching rows, along with all rows in the right table (whether there is a match or not). <br><br>
a FULL JOIN returns all rows from both tables. Note that in general, any row that does not have a match in both tables will have NULL entries for the missing values. <br><br>
If you would like to vertically concatenate columns, you can do so with a UNION. 

### Now we would like to tackle the US Traffic Fatality Records database that contains information on traffic accidents in the US where at least one person died.

In [28]:
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

traffic_dataset = client.get_dataset(traffic_ref)

traffic_tables = list(client.list_tables(traffic_dataset)) 

tabs = []
for table in traffic_tables: 
    tabs.append(table.table_id)

print(f"There are {len(tabs)} tables, and their names are: {tabs}")
    

There are 143 tables, and their names are: [' accident_2015', ' accident_2016', ' accident_2017', ' accident_2018', ' accident_2019', ' accident_2020', ' cevent_2015', ' cevent_2016', ' cevent_2017', ' cevent_2018', ' cevent_2019', ' cevent_2020', ' damage_2015', ' damage_2016', ' damage_2017', ' damage_2018', ' damage_2019', ' damage_2020', ' distract_2015', ' distract_2016', ' distract_2017', ' distract_2018', ' distract_2019', ' distract_2020', ' drimpair_2015', ' drimpair_2016', ' drimpair_2017', ' drimpair_2018', ' drimpair_2019', ' drimpair_2020', ' factor_2015', ' factor_2016', ' factor_2017', ' factor_2018', ' factor_2019', ' factor_2020', ' maneuver_2015', ' maneuver_2016', ' maneuver_2017', ' maneuver_2018', ' maneuver_2019', ' maneuver_2020', ' nmcrash_2015', ' nmcrash_2016', ' nmcrash_2017', ' nmcrash_2018', ' nmcrash_2019', ' nmcrash_2020', ' nmimpair_2015', ' nmimpair_2016', ' nmimpair_2017', ' nmimpair_2018', ' nmimpair_2019', ' nmimpair_2020', ' nmprior_2015', ' nmprior

In [29]:
table2015_ref = traffic_dataset.table("accident_2015")

table2015 = client.get_table(table2015_ref)

client.list_rows(table2015, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,30,Montana,300019,5,5,0,0,0,7,7,...,45,0,None,0,None,0,None,1,0,2015-03-28 14:58:00+00:00
1,39,Ohio,390099,7,7,0,0,0,15,15,...,24,27,Backup Due to Prior Crash,0,None,0,None,1,0,2015-02-14 11:19:00+00:00
2,49,Utah,490123,16,16,0,0,0,28,28,...,99,0,None,0,None,0,None,1,0,2015-04-14 12:24:00+00:00
3,48,Texas,481184,6,5,1,0,5,5,10,...,99,0,None,0,None,0,None,1,0,2015-05-27 16:40:00+00:00
4,41,Oregon,410333,11,11,0,0,0,14,14,...,99,0,None,0,None,0,None,1,0,2015-11-17 18:17:00+00:00


We would like to see which day of the week has the most fatal motor accidents

In [30]:
query3 = """
        select extract(dayofweek from timestamp_of_crash) as Day_of_week , count(consecutive_number) as Num_accidents
        from `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        group by Day_of_week
        order by Num_accidents Desc
        """


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

query_job = client.query(query3, job_config=safe_config)

accident_by_day = query_job.to_dataframe()

accident_by_day.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Day_of_week,Num_accidents
0,7,5659
1,1,5298
2,6,4916
3,5,4460
4,4,4182


##### In Bigquery, the dayofweek returns an integer from 1 to 7:<BR>
1: Sunday <BR>
2: Monday<BR>
3: Tuesday<BR>
4: Wednesday<BR>
5: Thursday<BR>
6: Friday<BR>
7: Saturday

Let's work with crypto_bitcoin dataset from bigquery public data

In [31]:
btc_ref = client.dataset("crypto_bitcoin", project="bigquery-public-data")

# API request - fetch the dataset
btc_data = client.get_dataset(btc_ref)

# Construct a reference to the "transactions" table
btc_table_ref = btc_data.table("transactions")

# API request - fetch the table
btc_table = client.get_table(btc_table_ref)

# Preview the first five lines of the "transactions" table
client.list_rows(btc_table, max_results=5).to_dataframe()

,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,275,275,1,0,00000000dc55860c8a29c58d45209318fa9e9dc2c1833a...,181,2009-01-12 06:02:13+00:00,2009-01-01,1,2,4000000000.000000000,4000000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'f4184...","[{'index': 0, 'script_asm': '04b5abd412d4341b4..."
1,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,275,275,1,0,0000000054487811fc4ff7a95be738aa5ad9320c394c48...,182,2009-01-12 06:12:16+00:00,2009-01-01,1,2,3000000000.000000000,3000000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'a16f3...","[{'index': 0, 'script_asm': '0401518fa1d1e1e3e..."
2,12b5633bad1f9c167d523ad1aa1947b2732a865bf5414e...,276,276,1,0,00000000f46e513f038baf6f2d9a95b2a28d8a6c985bcf...,183,2009-01-12 06:34:22+00:00,2009-01-01,1,2,2900000000.000000000,2900000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': '591e9...","[{'index': 0, 'script_asm': '04baa9d3665315562..."
3,828ef3b079f9c23829c56fe86e85b4a69d9e06e5b54ea5...,276,276,1,0,00000000fb5b44edc7a1aa105075564a179d65506e2bd2...,248,2009-01-12 20:04:20+00:00,2009-01-01,1,2,2800000000.000000000,2800000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': '12b56...","[{'index': 0, 'script_asm': '04bed827d37474bef..."
4,35288d269cee1941eaebb2ea85e32b42cdb2b04284a56d...,277,277,1,0,00000000689051c09ff2cd091cc4c22c10b965eb8db3ad...,545,2009-01-15 05:48:32+00:00,2009-01-01,1,2,2500000000.000000000,2500000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'd71fd...","[{'index': 0, 'script_asm': '044a656f065871a35..."


How many bitcoin transactions are made per month?

In [32]:
# with a CTE we convert the datetime format of the column to date format
btc_query = """
            WITH time AS 
            (
                SELECT DATE(block_timestamp) AS trans_date
                FROM `bigquery-public-data.crypto_bitcoin.transactions`
                WHERE _PARTITIONDATE >= '2023-01-01'
            )
            SELECT COUNT(1) AS transactions, trans_date
            FROM time
            GROUP BY trans_date
            ORDER BY trans_date
            """


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(btc_query, job_config=safe_config)

transactions_by_date = query_job.to_dataframe()

transactions_by_date.head()

BadRequest: 400 Unrecognized name: _PARTITIONDATE at [6:23]; reason: invalidQuery, location: query, message: Unrecognized name: _PARTITIONDATE at [6:23]

Location: US
Job ID: c128f3cf-86bd-4689-999a-1f03491db24a


Let's work with github_repos dataset from bigquery public data

In [ ]:
github_ref = client.dataset("github_repos", project="bigquery-public-data")

github_data = client.get_dataset(github_ref)

license_table_ref = github_data.table("licenses")

license_table = client.get_table(license_table_ref)

client.list_rows(license_table, max_results=5).to_dataframe()

,repo_name,license
0,autarch/Dist-Zilla-Plugin-Test-TidyAll,artistic-2.0
1,thundergnat/Prime-Factor,artistic-2.0
2,kusha-b-k/Turabian_Engin_Fan,artistic-2.0
3,onlinepremiumoutlet/onlinepremiumoutlet.github.io,artistic-2.0
4,huangyuanlove/LiaoBa_Service,artistic-2.0


In [ ]:
files_table_ref = github_data.table("sample_files")

files_table = client.get_table(files_table_ref)

client.list_rows(files_table, max_results=5).to_dataframe()

,repo_name,ref,path,mode,id,symlink_target
0,EOL/eol,refs/heads/master,generate/vendor/railties,40960,0338c33fb3fda57db9e812ac7de969317cad4959,/usr/share/rails-ruby1.8/railties
1,np/ling,refs/heads/master,tests/success/merger_seq_inferred.t/merger_seq...,40960,dd4bb3d5ecabe5044d3fa5a36e0a9bf7ca878209,../../../fixtures/all/merger_seq_inferred.ll
2,np/ling,refs/heads/master,fixtures/sequence/lettype.ll,40960,8fdf536def2633116d65b92b3b9257bcf06e3e45,../all/lettype.ll
3,np/ling,refs/heads/master,fixtures/failure/wrong_order_seq3.ll,40960,c2509ae1196c4bb79d7e60a3d679488ca4a753e9,../all/wrong_order_seq3.ll
4,np/ling,refs/heads/master,issues/sequence/keep.t,40960,5721de3488fb32745dfc11ec482e5dd0331fecaf,../keep.t


the mutual column in both tables is repo_name column


Next, we write a query that uses information in both tables to determine how many files are released in each license.

In [ ]:
repo_query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
            """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(repo_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
file_count_by_license = query_job.to_dataframe()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [ ]:
print(file_count_by_license)

         license  number_of_files
0            mit         20560894
1        gpl-2.0         16608922
2     apache-2.0          7201141
3        gpl-3.0          5107676
4   bsd-3-clause          3465437
5       agpl-3.0          1372100
6       lgpl-2.1           799664
7   bsd-2-clause           692357
8       lgpl-3.0           582277
9        mpl-2.0           457000
10       cc0-1.0           449149
11       epl-1.0           322255
12     unlicense           208602
13  artistic-2.0           147391
14           isc           118332


##### ANALYTIC FUNCTIONS <BR>
Let's work with the San Francisco Open Data dataset

In [33]:
sf_data_ref = client.dataset("san_francisco", project = "bigquery-public-data")

sf_data = client.get_dataset(sf_data_ref)

sf_bikeshare_table_ref = sf_data.table("bikeshare_trips")

sf_table_bikeshare = client.get_table(sf_bikeshare_table_ref)

client.list_rows(sf_table_bikeshare, max_results=5).to_dataframe()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


In [ ]:
# Query to count the (cumulative) number of trips per day (year 2015)

num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
                               ) AS cumulative_trips
                      FROM trips_by_day
                  """

# Run the query, and return a pandas DataFrame
num_trips_result = client.query(num_trips_query).result().to_dataframe()
num_trips_result.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,trip_date,num_trips,cumulative_trips
0,2015-01-01,181,181
1,2015-01-02,428,609
2,2015-01-03,283,892
3,2015-01-04,206,1098
4,2015-01-05,1186,2284


In [ ]:
# Query to track beginning and ending stations on October 25, 2015, for each bike

start_end_query = """
                  SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                      start_station_id,
                      end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25' 
                  """

# Run the query, and return a pandas DataFrame
start_end_result = client.query(start_end_query).result().to_dataframe()
start_end_result.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,bike_number,trip_time,first_station_id,last_station_id,start_station_id,end_station_id
0,22,13:25:00,2,16,2,16
1,25,11:43:00,77,51,77,60
2,25,12:14:00,77,51,60,51
3,29,14:59:00,46,74,46,60
4,29,21:23:00,46,74,60,74


In [ ]:
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

traffic_dataset = client.get_dataset(traffic_ref)

traffic_tables = list(client.list_tables(traffic_dataset)) 

tabs = []
for table in traffic_tables: 
    tabs.append(table.table_id)

print(f"There are {len(tabs)} tables, and their names are: {tabs}")
    

##### Nested and Repeated data <br>
Let's work with Google Analytics Sample dataset

In [36]:
gglanl_ref = client.dataset("google_analytics_sample", project = "bigquery-public-data")

gglanl_data = client.get_dataset(gglanl_ref)

gglanl_tables = list(client.list_tables(gglanl_data))

tabs = []
for table in gglanl_tables: 
    tabs.append(table.table_id)

print(f"There are {len(tabs)} tables, and their names are: {tabs}")

There are 369 tables, and their names are: ['Google-ecommerce-dataset-table', 'daily_total_visits', 'daily_visits', 'ga_sessions_20160801', 'ga_sessions_20160802', 'ga_sessions_20160803', 'ga_sessions_20160804', 'ga_sessions_20160805', 'ga_sessions_20160806', 'ga_sessions_20160807', 'ga_sessions_20160808', 'ga_sessions_20160809', 'ga_sessions_20160810', 'ga_sessions_20160811', 'ga_sessions_20160812', 'ga_sessions_20160813', 'ga_sessions_20160814', 'ga_sessions_20160815', 'ga_sessions_20160816', 'ga_sessions_20160817', 'ga_sessions_20160818', 'ga_sessions_20160819', 'ga_sessions_20160820', 'ga_sessions_20160821', 'ga_sessions_20160822', 'ga_sessions_20160823', 'ga_sessions_20160824', 'ga_sessions_20160825', 'ga_sessions_20160826', 'ga_sessions_20160827', 'ga_sessions_20160828', 'ga_sessions_20160829', 'ga_sessions_20160830', 'ga_sessions_20160831', 'ga_sessions_20160901', 'ga_sessions_20160902', 'ga_sessions_20160903', 'ga_sessions_20160904', 'ga_sessions_20160905', 'ga_sessions_2016090

We would like to work with the 'ga_sessions_20170801' table

In [37]:
# it is also possible to reference a table from the data reference and not the metadata of the dataset itself
# so to construct a reference for tables, having only the dataset reference can be sufficient and you don't have to get the dataset beforehands.

gglanl_table_ref = gglanl_ref.table("ga_sessions_20170801")
gglanl_table = client.get_table(gglanl_table_ref)

client.list_rows(gglanl_table, max_results=5).to_dataframe()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1501591568,1501591568,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",3418334011779872055,None,None,Organic Search,Not Socially Engaged
1,<NA>,2,1501589647,1501589647,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",2474397855041322408,None,None,Referral,Not Socially Engaged
2,<NA>,1,1501616621,1501616621,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",5870462820713110108,None,None,Referral,Not Socially Engaged
3,<NA>,1,1501601200,1501601200,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",9397809171349480379,None,None,Referral,Not Socially Engaged
4,<NA>,1,1501615525,1501615525,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6089902943184578335,None,None,Referral,Not Socially Engaged


In [41]:
print("SCHEMA field for the 'totals' column:\n")
print(gglanl_table.schema[5])

print("\nSCHEMA field for the 'device' column:\n")
print(gglanl_table.schema[7])

print("\nSCHEMA field for the 'hits' column:\n")
print(gglanl_table.schema[10])

SCHEMA field for the 'totals' column:

SchemaField('totals', 'RECORD', 'NULLABLE', None, None, (SchemaField('visits', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('hits', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('pageviews', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('timeOnSite', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('bounces', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('transactions', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('transactionRevenue', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('newVisits', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('screenviews', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('uniqueScreenviews', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('timeOnScreen', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('totalTransactionRevenue', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('sessionQu

We refer to the "browser" field (which is nested in the "device" column) and the "transactions" field (which is nested inside the "totals" column) as device.browser and totals.transactions

In [ ]:
# Query to count the number of transactions per browser
query_num_trns = """
        SELECT device.browser AS device_browser,
            SUM(totals.transactions) as total_transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
        GROUP BY device_browser
        ORDER BY total_transactions DESC
        """


num_trns = client.query(query_num_trns).result().to_dataframe()
num_trns.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,device_browser,total_transactions
0,Chrome,41
1,Safari,3
2,Firefox,1
3,Edge,<NA>
4,Coc Coc,<NA>


By storing the information in the "device" and "totals" columns as STRUCTs (as opposed to separate tables), we avoid expensive JOINs. This increases performance and keeps us from having to worry about JOIN keys (and which tables have the exact data we need).

Now we'll work with the "hits" column as an example of data that is both nested and repeated. Since:<br>

"hits" is a STRUCT (contains nested data) and is repeated,<br>
"hitNumber", "page", and "type" are all nested inside the "hits" column, and<br>
"pagePath" is nested inside the "page" field,

In [42]:
# Query to determine most popular landing point on the website
query_pop = """
        SELECT hits.page.pagePath as path,
            COUNT(hits.page.pagePath) as counts
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`, 
            UNNEST(hits) as hits
        WHERE hits.type="PAGE" and hits.hitNumber=1
        GROUP BY path
        ORDER BY counts DESC
        """

# Run the query, and return a pandas DataFrame
popular_landing = client.query(query_pop).result().to_dataframe()
popular_landing.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,path,counts
0,/home,1257
1,/google+redesign/shop+by+brand/youtube,587
2,/google+redesign/apparel/mens/mens+t+shirts,117
3,/signin.html,78
4,/basket.html,35
